# Anomaly Detection in Python

This notebook covers the material in the AI+ Training course on an introduction to fraud and anomaly detection. It goes through three main modules as follows:

1. Data Preparation
2. Probability and Statistical Approaches
3. Machine Learning Approaches

The following code loads all the necessary packages and libraries for the analysis.

In [9]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install scipy

Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install distfit

Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install benfordslaw

Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install distfit

Note: you may need to restart the kernel to use updated packages.


In [16]:
pip install benfordslaw

Note: you may need to restart the kernel to use updated packages.


In [17]:
import sklearn

In [18]:
import pandas as pd
import datetime as dt
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms
from distfit import distfit
from benfordslaw import benfordslaw
from sklearn.covariance import MinCovDet
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import OneClassSVM

## Data Preparation

This module details the area of data preparation for good anomaly detection. Anomaly detection is only as good as the data and the features that you have to detect anomalies. This module covers four main concepts in data preparation:

1. Feature Engineering
2. Recency and Frequency
3. Periodic Means
4. Categorical Feature Engineering

Let's load the dataset we will be working with.

In [19]:
ins = pd.read_csv("transactions_ins.csv")

ins.head()

,Date,Cust_ID,Transaction,Type,Reward_R,Reward_A,Cov_Limit,Income
0,9/10/1977,PSX000100006,IN,T,NaN,NaN,50000.0,61000.0
1,12/31/2005,PSX000100006,CL,T,NaN,NaN,50000.0,61000.0
2,1/17/2006,PSX000100006,RE,T,265.0,50000.0,NaN,NaN
3,12/15/1998,PSX000100010,IN,T,NaN,NaN,100000.0,29000.0
4,6/7/1961,PSX000100013,IN,V,NaN,NaN,100000.0,48000.0


In [20]:
ins.describe(include='all')

,Date,Cust_ID,Transaction,Type,Reward_R,Reward_A,Cov_Limit,Income
count,278877,278877,278877,278877,44488.000000,4.448800e+04,2.343890e+05,234389.000000
unique,23165,121191,4,3,NaN,NaN,NaN,NaN
top,9/15/2010,PSX000704539,IN,T,NaN,NaN,NaN,NaN
freq,44,21,121191,139958,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,315.732602,2.247246e+05,2.057786e+05,53363.899594
std,NaN,NaN,NaN,NaN,126.895957,2.284420e+05,1.984034e+05,25199.547677
min,NaN,NaN,NaN,NaN,100.000000,0.000000e+00,5.000000e+04,15000.000000
25%,NaN,NaN,NaN,NaN,207.000000,1.000000e+05,1.000000e+05,34422.000000
50%,NaN,NaN,NaN,NaN,314.000000,1.500000e+05,1.500000e+05,50000.000000
75%,NaN,NaN,NaN,NaN,421.000000,3.000000e+05,2.500000e+05,68026.000000


In [21]:
ins['Date'] = pd.to_datetime(ins['Date'])

### Feature Engineering

The concept of feature engineering is vitally important to anomaly detection. In my personal experience, the best features aren't just automatically generated by a computer, but thought up by a knowledgable individual working on the problem. 

We are going to role up the transactions to be one per individual. Here are some basic features when rolling up our transactions that might be important when discovering fraud:
- Final income
- Time between claim and reward
- Coverage limit to income ratio at claim

Think of some more of your own!

Let's create final income.

In [22]:
ins_income = ins[ins['Transaction'] == 'CL']

ins_income = ins_income[['Cust_ID', 'Cov_Limit', 'Income']]

ins_income.head(n = 10)

,Cust_ID,Cov_Limit,Income
1,PSX000100006,50000.0,61000.0
5,PSX000100013,100000.0,48000.0
14,PSX000100073,100000.0,35000.0
18,PSX000100081,50000.0,69000.0
24,PSX000100122,150000.0,105000.0
31,PSX000100146,100000.0,38000.0
50,PSX000100231,400000.0,79155.0
56,PSX000100236,250000.0,54738.0
62,PSX000100286,150000.0,95973.0
67,PSX00010030,350000.0,37725.0


Now, let's create time in between claim and reward.

In [23]:
ins_time = ins[(ins['Transaction'] == 'CL') | (ins['Transaction'] == 'RE')]

ins_time.head(n = 10)

,Date,Cust_ID,Transaction,Type,Reward_R,Reward_A,Cov_Limit,Income
1,2005-12-31,PSX000100006,CL,T,NaN,NaN,50000.0,61000.0
2,2006-01-17,PSX000100006,RE,T,265.0,50000.0,NaN,NaN
5,2001-06-12,PSX000100013,CL,V,NaN,NaN,100000.0,48000.0
6,2001-07-11,PSX000100013,RE,V,265.0,100000.0,NaN,NaN
14,2004-01-21,PSX000100073,CL,V,NaN,NaN,100000.0,35000.0
15,2004-02-14,PSX000100073,RE,V,450.0,100000.0,NaN,NaN
18,2009-06-28,PSX000100081,CL,T,NaN,NaN,50000.0,69000.0
19,2009-07-26,PSX000100081,RE,T,543.0,0.0,NaN,NaN
24,1991-08-23,PSX000100122,CL,T,NaN,NaN,150000.0,105000.0
25,1991-09-08,PSX000100122,RE,T,200.0,150000.0,NaN,NaN


In [28]:
ins_time['diff'] = ins_time['Date'] - ins_time['Date'].shift(1)

ins_time.head(n = 10)

,Date,Cust_ID,Transaction,Type,Reward_R,Reward_A,Cov_Limit,Income,diff
2,2006-01-17,PSX000100006,RE,T,265.0,50000.0,NaN,NaN,NaT
6,2001-07-11,PSX000100013,RE,V,265.0,100000.0,NaN,NaN,-1651 days
15,2004-02-14,PSX000100073,RE,V,450.0,100000.0,NaN,NaN,948 days
19,2009-07-26,PSX000100081,RE,T,543.0,0.0,NaN,NaN,1989 days
25,1991-09-08,PSX000100122,RE,T,200.0,150000.0,NaN,NaN,-6531 days
32,2009-04-21,PSX000100146,RE,T,275.0,100000.0,NaN,NaN,6435 days
51,2008-05-05,PSX000100231,RE,V,405.0,400000.0,NaN,NaN,-351 days
57,1970-10-06,PSX000100236,RE,T,469.0,250000.0,NaN,NaN,-13726 days
63,2003-03-20,PSX000100286,RE,V,337.0,150000.0,NaN,NaN,11853 days
68,1980-01-09,PSX00010030,RE,T,377.0,350000.0,NaN,NaN,-8471 days


In [29]:
ins_time = ins_time[ins_time['Transaction'] == 'RE']

ins_time = ins_time[['Date', 'Cust_ID', 'Type', 'Reward_R', 'Reward_A', 'diff']]

ins_time.head(n = 10)

,Date,Cust_ID,Type,Reward_R,Reward_A,diff
2,2006-01-17,PSX000100006,T,265.0,50000.0,NaT
6,2001-07-11,PSX000100013,V,265.0,100000.0,-1651 days
15,2004-02-14,PSX000100073,V,450.0,100000.0,948 days
19,2009-07-26,PSX000100081,T,543.0,0.0,1989 days
25,1991-09-08,PSX000100122,T,200.0,150000.0,-6531 days
32,2009-04-21,PSX000100146,T,275.0,100000.0,6435 days
51,2008-05-05,PSX000100231,V,405.0,400000.0,-351 days
57,1970-10-06,PSX000100236,T,469.0,250000.0,-13726 days
63,2003-03-20,PSX000100286,V,337.0,150000.0,11853 days
68,1980-01-09,PSX00010030,T,377.0,350000.0,-8471 days


Lastly, let's combine our datasets from above and create coverage limit to income ratio.

In [25]:
ins_feat = ins_time.merge(ins_income)

ins_feat.head(n = 10)

,Date,Cust_ID,Transaction,Type,Reward_R,Reward_A,Cov_Limit,Income


In [26]:
ins_feat['Cov_Income_Ratio'] = ins_feat['Cov_Limit'] / ins_feat['Income']

ins_feat.head(n = 10)

,Date,Cust_ID,Transaction,Type,Reward_R,Reward_A,Cov_Limit,Income,Cov_Income_Ratio


### Recency and Frequency

We will look at recency and frequency one at a time. Let's start with recency. For ease of application, let's use all the differences in time for all customers to build our recency exponential distribution instead of building one per customer.

Specifically, we will look at recency of changes in policy.

In [ ]:
ins_rec = ins

ins_rec['diff'] = ins_rec['Date'] - ins_rec['Date'].shift(1)

ins_rec = ins_rec[ins_rec['Transaction'] == 'CH']

ins_rec.head(n = 10)

In [ ]:
exp_rec = st.distributions.expon.fit(ins_rec['diff'].dt.days)

print(exp_rec)

In [ ]:
ins_rec['rec'] = np.exp(-(1/exp_rec[1])*(ins_rec['diff'].dt.days))

ins_rec.head(n = 10)

In [ ]:
ins_rec = ins_rec.groupby('Cust_ID', as_index=False)['rec'].mean()

ins_rec.head(n = 10)

For our data, frequency of changes in policy might also be a flag of fraud. 

In [ ]:
ins_changes = ins

ins_changes['T_CH'] = pd.get_dummies(ins_changes['Transaction'], prefix='T')['T_CH']

ins_changes = ins_changes.groupby('Cust_ID', as_index=False)['T_CH'].sum()

ins_changes.head(n = 10)

Let's combine these recency and frequency features into our features dataset.

In [ ]:
ins_feat = ins_feat.merge(ins_rec)

ins_feat.head(n = 10)

In [ ]:
ins_feat = ins_feat.merge(ins_changes)

ins_feat.head(n = 10)

### Periodic Mean

Python isn't as well created for this feature as compared to R.

### Categorical Feature Engineering

Feature engineering doesn't stop with just continuous variables in transactional data. We need to account for categorical features as well.

In our data, the reward reason (Reward_R) variable is categorical. It has numerical representations for reasons of passing that are given on the life insurance policy. Some of these reasons are approved - life insurance policy is paid out. However, not all reasons of passing are covered so the policy isn't paid out in those scenarios.

In [ ]:
len(set(ins_feat['Reward_R']))

Instead of having 480 possible categories, let combine these into two groups - approved and rejected reasons - based on the reward amount. If the reward amount was 0, then the reason was not covered by the policy.

In [ ]:
ins_feat['Reward_Y'] = ins_feat['Reward_A'].apply(lambda x: 'Y' if x > 0 else 'N')

ins_feat.head(n = 10)

## Probability and Statistical Approaches

This module details the area of probability and statistical approaches to anomaly detection. Classical probability and statistical approaches to anomaly detection are a great foundation and are still widely used and useful in detecting anomalies. This module covers four main concepts in probability and statistical approaches to anomaly detection:

1. Benford's Law
2. Z-scores and Robust Z-scores
3. IQR Rule and It's Adjustment
4. Mahalanobis Distances

### Benford's Law

We don't have addresses in our data so it makes it hard to see if someone applied for a life insurance policy using fake information. However, let's see how we would apply that to data. 

In [ ]:
bl = benfordslaw(alpha=0.05)

results = bl.fit(ins_feat['Income'])

bl.plot

### Z-Scores and Robust Z-scores

We want to evaluate if we have an observation far away from "normal" in our coverage to income ratio at time of claim. We are looking for really large life insurance coverage limits when income is not very high - a large ratio between the two.

In [ ]:
plt.hist(ins_feat['Cov_Income_Ratio'])
plt.xlabel('Ratio')
plt.ylabel('Frequency')
plt.title('Coverage to Income Ratio')

In [ ]:
ins_feat['Z_Cov_Income_Ratio'] = abs((ins_feat['Cov_Income_Ratio'] - ins_feat['Cov_Income_Ratio'].mean())/ins_feat['Cov_Income_Ratio'].std())

plt.hist(ins_feat['Z_Cov_Income_Ratio'])
plt.xlabel('Z-Scores')
plt.ylabel('Frequency')
plt.title('Coverage to Income Ratio Z-Score')

In [ ]:
len(ins_feat[ins_feat['Z_Cov_Income_Ratio'] > 3])

Let's switch to using the robust z-score calculation instead.

In [ ]:
ins_feat['RZ_Cov_Income_Ratio'] = abs((ins_feat['Cov_Income_Ratio'] - ins_feat['Cov_Income_Ratio'].median())/st.median_absolute_deviation(ins_feat['Cov_Income_Ratio']))

plt.hist(ins_feat['RZ_Cov_Income_Ratio'])
plt.xlabel('Robust Z-Scores')
plt.ylabel('Frequency')
plt.title('Coverage to Income Ratio Robust Z-Score')

In [ ]:
len(ins_feat[ins_feat['RZ_Cov_Income_Ratio'] > 3])

### IQR Rule and It's Adjustment

Another univariate approach to looking for outliers is the IQR Rule. However, the IQR Rule really works best for symmetric distributions. For our coverage limit to income ratio variable we know this isn't symmetric based on our above plots 

In [ ]:
fig1, ax1 = plt.subplots()
ax1.set_title('Coverage to Income Ratio')
ax1.boxplot(ins_feat['Cov_Income_Ratio'])

### Mahalanobis Distances

When looking at two or more variables at the same time, we need to account for multiple dimensions. Mahalanobis distances are a multivariate version of the z-score. 

Let compare the two variables of average recency of changes in policies and frequency of these changes.

In [ ]:
plt.scatter(x = ins_feat['Income'], y = ins_feat['Cov_Income_Ratio'], alpha = 0.5)
plt.xlabel('Income ($)')
plt.ylabel('Coverage to Income Ratio')
plt.title('Income vs. Coverage to Income Ratio')

In [ ]:
def confidence_ellipse(x, y, ax, n_std=2.0, facecolor='none', **kwargs):
    """
    Create a plot of the covariance confidence ellipse of *x* and *y*.

    Parameters
    ----------
    x, y : array-like, shape (n, )
        Input data.

    ax : matplotlib.axes.Axes
        The axes object to draw the ellipse into.

    n_std : float
        The number of standard deviations to determine the ellipse's radiuses.

    **kwargs
        Forwarded to `~matplotlib.patches.Ellipse`

    Returns
    -------
    matplotlib.patches.Ellipse
    """
    if x.size != y.size:
        raise ValueError("x and y must be the same size")

    cov = np.cov(x, y)
    pearson = cov[0, 1]/np.sqrt(cov[0, 0] * cov[1, 1])
    # Using a special case to obtain the eigenvalues of this
    # two-dimensionl dataset.
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = Ellipse((0, 0), width=ell_radius_x * 2, height=ell_radius_y * 2,
                      facecolor=facecolor, **kwargs)

    # Calculating the stdandard deviation of x from
    # the squareroot of the variance and multiplying
    # with the given number of standard deviations.
    scale_x = np.sqrt(cov[0, 0]) * n_std
    mean_x = np.mean(x)

    # calculating the stdandard deviation of y ...
    scale_y = np.sqrt(cov[1, 1]) * n_std
    mean_y = np.mean(y)

    transf = transforms.Affine2D() \
        .rotate_deg(45) \
        .scale(scale_x, scale_y) \
        .translate(mean_x, mean_y)

    ellipse.set_transform(transf + ax.transData)
    return ax.add_patch(ellipse)

In [ ]:
fig, ax_nstd = plt.subplots(figsize=(6, 6))

mu = ins_feat['Income'].mean(), ins_feat['Cov_Income_Ratio'].mean()

ax_nstd.axvline(c='grey', lw=1)
ax_nstd.axhline(c='grey', lw=1)

x = ins_feat['Income']
y = ins_feat['Cov_Income_Ratio']
ax_nstd.scatter(x, y, s=0.5)

confidence_ellipse(x, y, ax_nstd, n_std=4.5, edgecolor='blue', linestyle='-')

ax_nstd.scatter(mu[0], mu[1], c='red', s=3)
ax_nstd.set_title('Income vs. Coverage to Income Ratio')
plt.show()

In [ ]:
d = {'Income': ins_feat['Income'], 'CIRatio': ins_feat['Cov_Income_Ratio']}
df = pd.DataFrame(data=d)

covMCD = MinCovDet(random_state=0).fit(df)

In [ ]:
def confidence_ellipse_r(x, y, ax, n_std=4.5, facecolor='none', **kwargs):
    """
    Create a plot of the covariance confidence ellipse of *x* and *y*.

    Parameters
    ----------
    x, y : array-like, shape (n, )
        Input data.

    ax : matplotlib.axes.Axes
        The axes object to draw the ellipse into.

    n_std : float
        The number of standard deviations to determine the ellipse's radiuses.

    **kwargs
        Forwarded to `~matplotlib.patches.Ellipse`

    Returns
    -------
    matplotlib.patches.Ellipse
    """
    if x.size != y.size:
        raise ValueError("x and y must be the same size")
        
    d = {'X': x, 'Y': y}
    df = pd.DataFrame(data=d)

    covMCD = MinCovDet(random_state=0).fit(df)

    cov = covMCD.covariance_
    pearson = cov[0, 1]/np.sqrt(cov[0, 0] * cov[1, 1])
    # Using a special case to obtain the eigenvalues of this
    # two-dimensionl dataset.
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = Ellipse((0, 0), width=ell_radius_x * 2, height=ell_radius_y * 2,
                      facecolor=facecolor, **kwargs)

    # Calculating the stdandard deviation of x from
    # the squareroot of the variance and multiplying
    # with the given number of standard deviations.
    scale_x = np.sqrt(cov[0, 0]) * n_std
    mean_x = covMCD.location_[0]

    # calculating the stdandard deviation of y ...
    scale_y = np.sqrt(cov[1, 1]) * n_std
    mean_y = covMCD.location_[1]

    transf = transforms.Affine2D() \
        .rotate_deg(45) \
        .scale(scale_x, scale_y) \
        .translate(mean_x, mean_y)

    ellipse.set_transform(transf + ax.transData)
    return ax.add_patch(ellipse)

In [ ]:
fig, ax_nstd = plt.subplots(figsize=(6, 6))

mu = ins_feat['Income'].mean(), ins_feat['Cov_Income_Ratio'].mean()

ax_nstd.axvline(c='grey', lw=1)
ax_nstd.axhline(c='grey', lw=1)

x = ins_feat['Income']
y = ins_feat['Cov_Income_Ratio']
ax_nstd.scatter(x, y, s=0.5)

confidence_ellipse(x, y, ax_nstd, n_std=4.5, edgecolor='blue', linestyle='-')
confidence_ellipse_r(x, y, ax_nstd, n_std=4.5, edgecolor='red', linestyle='-')

ax_nstd.scatter(mu[0], mu[1], c='red', s=3)
ax_nstd.set_title('Income vs. Coverage to Income Ratio')
plt.show()

## Machine Learning Approaches

This module details the area of machine learning approaches to anomaly detection. With the enhancements and creation of machine learning approaches to modeling, these same techniques form the foundation of more advanced anomaly detection methods. This module covers five main concepts in machine learning approaches to anomaly detection:

1. k-Nearest Neighbors (k-NN)
2. Local Outlier Factor (LOF)
3. Isolation Forests
4. Classifier-Adjusted Density Estimation (CADE)
5. One-Class Support Vector Machines (SVM)

### k-Nearest Neighbors

Instead of looking at more statistical approaches, we can directly look at the average distance each point is from its k-nearest neighbors.

In [ ]:
d = {'Income': ins_feat['Income'], 'CIRatio': ins_feat['Cov_Income_Ratio']}
df = pd.DataFrame(data=d)

nbrs = NearestNeighbors(n_neighbors=6, algorithm='ball_tree').fit(df)
distances, indices = nbrs.kneighbors(df)

print(distances)
print(indices)

In [ ]:
df.to_csv('')

In [ ]:
df['nndist'] = np.mean(distances, axis = 1)

plt.scatter(x = ins_feat['Income'], y = ins_feat['Cov_Income_Ratio'], alpha = 0.5, s = (df['nndist'])**(0.75))
plt.xlabel('Income')
plt.ylabel('Coverage to Income Ratio (Thousands $)')
plt.title('Income vs. Coverage to Income Ratio')

### Local Outlier Factor (LOF)

Let's examine more local outliers.

In [ ]:
d = {'Income': ins_feat['Income'], 'CIRatio': ins_feat['Cov_Income_Ratio']}
df = pd.DataFrame(data=d)

lof = LocalOutlierFactor(n_neighbors=6)
lof.fit_predict(df)

df['lof'] = -lof.negative_outlier_factor_

plt.scatter(x = ins_feat['Income'], y = ins_feat['Cov_Income_Ratio'], alpha = 0.5, s = df['lof']*10)
plt.xlabel('Income')
plt.ylabel('Coverage to Income Ratio (Thousands $)')
plt.title('Income vs. Coverage to Income Ratio')

In [ ]:
plt.hist(df['lof']*10)

### Isolation Forest

Instead of distance based measures, now we switch to tree based approaches.

In [ ]:
d = {'Income': ins_feat['Income'], 'CIRatio': ins_feat['Cov_Income_Ratio']}
df = pd.DataFrame(data=d)

isofor = IsolationForest(random_state=0, n_estimators = 500).fit(df)
iso = -isofor.score_samples(df)

df['iso'] = iso
df['iso2'] = ((iso - np.min(iso))/(np.max(iso) - np.min(iso))*10)**2

plt.scatter(x = ins_feat['Income'], y = ins_feat['Cov_Income_Ratio'], alpha = 0.25, s = df['iso2'])
plt.xlabel('Income ($)')
plt.ylabel('Coverage to Income Ratio')
plt.title('Income vs. Coverage to Income Ratio')

In [ ]:
plt.hist(df['iso'])

### Classifier-Adjusted Density Estimation (CADE)

Let's now finish off with more density based approaches to anomaly detection. Unfortunately, since CADE is so new we need to write it ourselves.

In [ ]:
d = {'Income': ins_feat['Income'], 'CIRatio': ins_feat['Cov_Income_Ratio'], 'Target': 0}
df = pd.DataFrame(data=d)

df.head(n = 10)

In [ ]:
d_fake = {'Income': np.random.uniform(np.min(ins_feat['Income']), np.max(ins_feat['Income']), size = len(ins_feat['Income'])), 
          'CIRatio': np.random.uniform(np.min(ins_feat['Cov_Income_Ratio']), np.max(ins_feat['Cov_Income_Ratio']), 
                                       size = len(ins_feat['Cov_Income_Ratio'])),
          'Target': 1}
df_fake = pd.DataFrame(data=d_fake)

df_fake.head(n = 10)

In [ ]:
df_comb = df.append(df_fake, ignore_index=True)

print(df_comb)

In [ ]:
model = RandomForestClassifier()
model.fit(df_comb[['Income', 'CIRatio']], df_comb['Target'])

df['pred'] = model.predict_proba(df[['Income', 'CIRatio']])[:,1]
df['odds'] = df['pred'] / (1 - df['pred'])

df.head(n = 10)

In [ ]:
plt.hist(df['odds']*100)

In [ ]:
plt.scatter(x = ins_feat['Income'], y = ins_feat['Cov_Income_Ratio'], alpha = 0.5, s = df['odds']*100)
plt.xlabel('Income')
plt.ylabel('Coverage to Income Ratio (Thousands $)')
plt.title('Income vs. Coverage to Income Ratio')

### One-Class Support Vector Machines

Lastly, let's look at the support vector machine approach.

In [ ]:
d = {'Income': ins_feat['Income'], 'CIRatio': ins_feat['Cov_Income_Ratio']}
df = pd.DataFrame(data=d)

svm = OneClassSVM(gamma=0.00001, nu = 0.05).fit_predict(df)

In [ ]:
df['svm'] = svm

fig, ax = plt.subplots()

colors = {-1:'red', 1:'black'}

ax.scatter(ins_feat['Income'], ins_feat['Cov_Income_Ratio'], c = df['svm'].map(colors))
plt.xlabel('Income')
plt.ylabel('Coverage to Income Ratio (Thousands $)')
plt.title('Income vs. Coverage to Income Ratio')
plt.show()

In [ ]:
d = {'Income': ins_feat['Income'], 'CIRatio': ins_feat['Cov_Income_Ratio'], 'Target': 0}
df = pd.DataFrame(data=d)

d_fake = {'Income': np.random.uniform(np.min(ins_feat['Income']), np.max(ins_feat['Income']), size = len(ins_feat['Income'])), 
          'CIRatio': np.random.uniform(np.min(ins_feat['Cov_Income_Ratio']), np.max(ins_feat['Cov_Income_Ratio']), 
                                       size = len(ins_feat['Cov_Income_Ratio'])),
          'Target': 1}
df_fake = pd.DataFrame(data=d_fake)

df_comb = df.append(df_fake, ignore_index=True)

model = RandomForestClassifier()
model.fit(df_comb[['Income', 'CIRatio']], df_comb['Target'])

df['pred'] = model.predict_proba(df[['Income', 'CIRatio']])[:,1]